# JetBrains AI Assistant MCP Client Registry

This notebook demonstrates how to use a registry service to manage Model Context Protocol (MCP) server configurations at runtime for the [JetBrains AI Assistant](https://plugins.jetbrains.com/plugin/22282-jetbrains-ai-assistant) plugin. It shows how to add, update, and remove configurations that control how individual MCP servers are launched.

## What this demo shows

- How to get the registry service instance
- How to register a new MCP server configuration
- How to update an existing configuration
- How to remove a configuration

In [1]:
%use intellij-platform

IntelliJ Platform integration is loaded

Load the JetBrains AI Assistant bundled plugin using its `com.intellij.ml.llm` ID.

In [2]:
loadPlugins("com.intellij.ml.llm")

In [15]:
import com.intellij.ml.llm.mcp.client.McpClientRegistry
import com.intellij.ml.llm.mcp.client.McpClientRegistryException
import com.intellij.ml.llm.mcp.client.McpServerCommandDescriptor

val registry = McpClientRegistry.getInstance()
var id: Int = -1

try {
    // 1) Create and register a new MCP server configuration
    // The configuration is persisted and created in a disabled state.
    // A notification will be shown to the user,
    // and the user will be able to enable the server via the notification action.
    val initialDescriptor = McpServerCommandDescriptor(
        name = "demo-mcp-server",
        command = "npx",
        arguments = listOf("@modelcontextprotocol/new-server"),
        environmentVariables = emptyMap(),
        workingDirectory = "" // optional working directory, leave empty to inherit project dir or process default
    )

    // Optionally pass `project` to register a project-level configuration (instead of application-level)
    id = registry.addConfiguration(initialDescriptor, /*project = project */)
}
catch (e: McpClientRegistryException) {
    // Handle registry-specific issues (e.g., duplicate names, invalid id)
    // You may log the error, show a notification, or surface it to the user as appropriate.
}

In [14]:
try {
    // 2) Update the configuration if you need to change the startup command or its arguments
    val updatedDescriptor = McpServerCommandDescriptor(
        name = "demo-mcp-server",
        command = "npx",
        arguments = listOf("@modelcontextprotocol/new-server", "additional_argument"),
        environmentVariables = emptyMap(),
        workingDirectory = ""
    )
    registry.updateConfiguration(id, updatedDescriptor)
}
catch (e: McpClientRegistryException) {
}

In [12]:
try {
    // 3) Remove the configuration when it's no longer needed
    registry.removeConfiguration(id)
}
catch (e: McpClientRegistryException) {
}

## API reference: MCP registry service and descriptors

Public interfaces and data types for registering, updating, and removing MCP server configurations across application- and project-level scopes. This API is experimental.

```kotlin
package com.intellij.ml.llm.mcp.client

import com.intellij.openapi.application.ApplicationManager
import com.intellij.openapi.components.service
import com.intellij.openapi.project.Project
import org.jetbrains.annotations.ApiStatus

/**
 * An application service for managing Model Context Protocol (MCP) client configurations.
 *
 * Provides CRUD operations on persisted MCP server configurations for either:
 * - application-level scope when [project] is null, or
 * - project-level scope when a [Project] is provided.
 *
 * This API is experimental and may change in future versions.
 */
@ApiStatus.Experimental
interface McpClientRegistry {
  companion object {
    @JvmStatic
    fun getInstance(): McpClientRegistry = ApplicationManager.getApplication().service()
  }

  /**
   * Returns all persisted MCP server configurations for the given scope.
   *
   * @param project target project for project-level configurations; pass null for application-level configurations
   * @return a map of configuration id to [McpServerDescriptor]
   */
  fun getConfigurations(project: Project? = null): Map<Int, McpServerDescriptor>

  /**
   * Adds a new MCP server configuration to the specified scope.
   *
   * The configuration is persisted and created in the disabled state. If a configuration with the same name
   * already exists in the scope, the operation fails.
   *
   * @param descriptor server descriptor to persist
   * @param project target project for project-level persistence; pass null for application-level persistence
   * @return the generated unique configuration id
   * @throws McpClientRegistryException if validation fails or a configuration with the same name already exists
   */
  @Throws(McpClientRegistryException::class)
  fun addConfiguration(descriptor: McpServerDescriptor, project: Project? = null): Int

  /**
   * Updates an existing MCP server configuration in the specified scope.
   *
   * If the configuration type differs from the existing one, an exception is thrown and no changes are applied.
   *
   * @param id id of the configuration to update
   * @param descriptor new values for the configuration
   * @param project target project for project-level scope; pass null for application-level scope
   * @throws McpClientRegistryException if the configuration with the given id does not exist, validation fails,
   * or the descriptor type differs from the existing configuration type
   */
  @Throws(McpClientRegistryException::class)
  fun updateConfiguration(id: Int, descriptor: McpServerDescriptor, project: Project? = null)

  /**
   * Removes an existing MCP server configuration from the specified scope.
   *
   * @param id id of the configuration to remove
   * @param project target project for project-level scope; pass null for application-level scope
   * @throws McpClientRegistryException if the configuration with the given id does not exist
   */
  @Throws(McpClientRegistryException::class)
  fun removeConfiguration(id: Int, project: Project? = null)
}
```

`McpServerDescriptor`:
```kotlin
package com.intellij.ml.llm.mcp.client

import org.jetbrains.annotations.ApiStatus

@ApiStatus.Experimental
sealed class McpServerDescriptor(open val name: String)

@ApiStatus.Experimental
data class McpServerCommandDescriptor(
  override val name: String,
  val command: String,
  val arguments: List<String>,
  val environmentVariables: Map<String, String>,
  val workingDirectory: String,
) : McpServerDescriptor(name)

@ApiStatus.Experimental
data class McpServerUrlDescriptor(
  override val name: String,
  val url: String,
  val headers: Map<String, String>?,
) : McpServerDescriptor(name)
```